In [28]:
# https://www.youtube.com/watch?v=mdIvxFKc8-g
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizerFast, DistilBertModel, TFDistilBertForSequenceClassification

In [2]:
ds = tfds.load("amazon_us_reviews/Mobile_Electronics_v1_00", split = "train", shuffle_files = True)
df = tfds.as_dataframe(ds)

df.head()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\laksh\tensorflow_datasets\amazon_us_reviews\Mobile_Electronics_v1_00\0.1.0.incomplete8LC0E7…

Dataset amazon_us_reviews downloaded and prepared to C:\Users\laksh\tensorflow_datasets\amazon_us_reviews\Mobile_Electronics_v1_00\0.1.0. Subsequent calls will reuse this data.


,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1
4,b'2663569',0,b'US',b'Mobile_Electronics',b'B00GHZS4SC',b'350592810',b'HDE Multifunctional Bluetooth FM Audio Car K...,"b""it works but it has really bad sound quality...",b'2014-12-31',b'bad sound quality',b'R2PEOEUR1LP0GH',3,0,0,1


In [4]:
df.to_csv("Amazon_Mobile_reviews.csv")

In [7]:
df['sentiment'] = df["data/star_rating"].apply(lambda score : "Positive" if score >= 3 else "Negative")
df["sentiment"] = df["sentiment"].map({"Positive" : 1, "Negative" : 0})

df["Review_text"] = df["data/review_body"].str.decode("utf-8")
df[["Review_text", "sentiment"]].head()

,Review_text,sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1
4,it works but it has really bad sound quality. ...,1


In [38]:
df["sentiment"].value_counts()

1    38038
0    11962
Name: sentiment, dtype: int64

In [8]:
n = 54975
df.drop(df.tail(n).index, inplace = True)

In [9]:
reviews = df["Review_text"].values.tolist()
labels = df["sentiment"].values.tolist()

In [17]:
from sklearn.model_selection import train_test_split

train_sentence, valid_sentence, train_labels, valid_labels = train_test_split(reviews, labels, test_size = 0.2)

In [25]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [26]:
train_encoding = tokenizer(train_sentence, truncation=True, padding=True, max_length=128)
valid_encoding = tokenizer(valid_sentence, truncation=True, padding=True, max_length=128)

In [27]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoding), train_labels))
valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_encoding), valid_labels))

In [29]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", output_attentions=True)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i